In [2]:
import subprocess
from tqdm.notebook import tqdm

import pandas as pd
df = pd.read_csv('/data/small_instances.csv', sep='|', header=None)
list_instances = df[0].unique().tolist()

pbar = tqdm(total=6*len(list_instances))
maxes = {}
for instance in list_instances:
    maxes[instance] = (0, 0)
    for k in [1, 2, 3]:
        for m in [1, 2, 3]:
            if k < m: continue

            out = subprocess.Popen(
                list(map(str, ['/app/instance_evaluator', k, m, instance])),
                stdout=subprocess.PIPE, stderr=subprocess.STDOUT
            )
            stdout,stderr = out.communicate()
            assert out.returncode == 0
            
            if 'success' in stdout.decode().lower():
                if (k, m) > maxes[instance]:
                    maxes[instance] = (k, m)        
            pbar.update(1)
pbar.close()

  0%|          | 0/22782 [00:00<?, ?it/s]

In [6]:
next(i for i, v in maxes.items() if 0 in v)

StopIteration: 

In [16]:
pd.DataFrame([{'instance': i, 'kcmc': f'(K{v[0]}M{v[1]})'} for i, v in maxes.items()]
            ).to_csv('/data/tstr.small_instances.csv', sep='|', header=None, index=None)